In [3]:
import cv2
import numpy as np
import tensorflow as tf
tflite = tf.lite
from math import atan2, degrees

# -------------------------------
# Utility functions
# -------------------------------
def draw_keypoints(frame, keypoints, threshold=0.3):
    h, w, _ = frame.shape
    for y, x, score in keypoints:
        if score > threshold:
            cv2.circle(frame, (int(x * w), int(y * h)), 5, (0, 255, 0), -1)

def draw_connections(frame, keypoints, edges, threshold=0.3):
    h, w, _ = frame.shape
    for (start, end) in edges:
        y1, x1, s1 = keypoints[start]
        y2, x2, s2 = keypoints[end]
        if s1 > threshold and s2 > threshold:
            cv2.line(frame, (int(x1*w), int(y1*h)), (int(x2*w), int(y2*h)), (0,255,255), 2)

def calculate_angle(a, b):
    """Calculate angle between two points (a,b) in degrees relative to vertical"""
    dy = a[1] - b[1]
    dx = a[0] - b[0]
    return degrees(atan2(dy, dx))

# -------------------------------
# Load TFLite MoveNet model
# -------------------------------
interpreter = tflite.Interpreter(model_path="movenet-tflite-singlepose-lightning-v1.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# -------------------------------
# Define skeleton edges
# -------------------------------
# Keypoint indices from MoveNet
# 0: nose, 1:left_eye, 2:right_eye, 3:left_ear, 4:right_ear
# 5:left_shoulder, 6:right_shoulder, 7:left_elbow, 8:right_elbow
# 9:left_wrist, 10:right_wrist, 11:left_hip, 12:right_hip
# 13:left_knee, 14:right_knee, 15:left_ankle, 16:right_ankle
EDGES = [
    (0, 1), (0, 2), (1, 3), (2, 4),
    (0, 5), (0, 6),
    (5, 7), (7, 9),
    (6, 8), (8,10),
    (5,11), (6,12),
    (11,12),
    (11,13), (13,15),
    (12,14), (14,16)
]

# -------------------------------
# Capture video
# -------------------------------
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (192,192))
    input_image = np.expand_dims(img_resized.astype(np.float32), axis=0)

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], input_image)
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])[0][0]

    # keypoints_with_scores: 17x3 (y, x, score)
    draw_connections(frame, keypoints_with_scores, EDGES)
    draw_keypoints(frame, keypoints_with_scores)

    # -------------------------------
    # Posture detection: neck & torso angle
    # -------------------------------
    # Approximate points
    nose = keypoints_with_scores[0][:2]
    left_shoulder = keypoints_with_scores[5][:2]
    right_shoulder = keypoints_with_scores[6][:2]
    left_hip = keypoints_with_scores[11][:2]
    right_hip = keypoints_with_scores[12][:2]

    # Midpoints
    shoulder_mid = ((left_shoulder[0]+right_shoulder[0])/2, (left_shoulder[1]+right_shoulder[1])/2)
    hip_mid = ((left_hip[0]+right_hip[0])/2, (left_hip[1]+right_hip[1])/2)

    # Torso angle
    torso_angle = calculate_angle(hip_mid, shoulder_mid)  # relative to vertical

    # Neck angle (shoulder -> nose)
    neck_angle = calculate_angle(shoulder_mid, nose)

    cv2.putText(frame, f"Torso angle: {torso_angle:.1f} deg", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
    cv2.putText(frame, f"Neck angle: {neck_angle:.1f} deg", (10,60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)

    # -------------------------------
    cv2.imshow("Posture Tracking", frame)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
